# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [110]:
pokemon = pd.read_csv("Pokemon.csv")
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Column1,Column2,Column3
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,318,0,Match
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,405,0,Match
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,525,0,Match
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,625,0,Match
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,309,0,Match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,600,0,Match
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,700,0,Match
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True,600,0,Match
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True,680,0,Match


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [ ]:
# The total is calculated based on the sum of the following parameters: HP, Attack, Defense, Sp. Atk, Sp. Def, and Speed.
# I have checked the entire dataset and seems as if the formula worked for all pokemon



## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [103]:
dummies = pd.get_dummies(pokemon['Type 1'])
dummies.to_csv("dummies.csv")
dummies

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
798,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [109]:
pokemon_dummies = pokemon.join(dummies)
pokemon_dummies

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,0,1,0,0,0,0,0,0,0,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,0,1,0,0,0,0,0,0,0,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,0,1,0,0,0,0,0,0,0,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,...,0,1,0,0,0,0,0,0,0,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,...,0,0,0,0,0,0,0,1,0,0
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,...,0,0,0,0,0,0,0,1,0,0
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,...,0,0,0,0,0,0,1,0,0,0
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,...,0,0,0,0,0,0,1,0,0,0


In [130]:
correlation_table = pokemon_dummies.corr().sort_values("Total",ascending=False)
correlation_table

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
Column1,0.119813,1.000000,0.618748,0.736211,0.612787,0.747250,0.717609,0.575943,0.048384,0.501758,...,0.007594,-0.036057,0.004082,-0.002412,-0.104150,-0.057123,0.094364,0.037524,0.082000,-0.015640
Total,0.119813,1.000000,0.618748,0.736211,0.612787,0.747250,0.717609,0.575943,0.048384,0.501758,...,0.007594,-0.036057,0.004082,-0.002412,-0.104150,-0.057123,0.094364,0.037524,0.082000,-0.015640
Sp. Atk,0.088759,0.747250,0.362380,0.396362,0.223549,1.000000,0.506121,0.473018,0.036437,0.448907,...,0.040721,0.044316,-0.102064,0.025392,-0.194274,-0.072164,0.216686,-0.069929,-0.030298,0.024583
Attack,0.102298,0.736211,0.422386,1.000000,0.438687,0.396362,0.263990,0.381240,0.051451,0.345408,...,-0.032849,-0.055245,0.105399,-0.033892,-0.063720,-0.025379,-0.064427,0.103101,0.078949,-0.060321
Sp. Def,0.085817,0.717609,0.378718,0.263990,0.510747,0.506121,1.000000,0.259133,0.028486,0.363937,...,0.033514,-0.016411,-0.067175,0.027754,-0.109867,-0.051424,0.143193,0.031009,0.058646,-0.020088
HP,0.097614,0.618748,1.000000,0.422386,0.239622,0.362380,0.378718,0.175952,0.058683,0.273620,...,-0.038565,-0.024116,0.036175,0.018891,0.117377,-0.014991,0.014900,-0.036824,-0.029563,0.044330
Defense,0.094786,0.612787,0.239622,0.438687,1.000000,0.223549,0.510747,0.015227,0.042419,0.246377,...,0.048110,-0.030232,0.072058,-0.013689,-0.167796,-0.030684,-0.054733,0.208650,0.315013,-0.011601
Speed,0.010733,0.575943,0.175952,0.381240,0.015227,0.473018,0.259133,1.000000,-0.023121,0.326715,...,-0.027648,-0.067695,-0.030723,-0.029182,0.042114,-0.030860,0.126019,-0.102742,-0.083775,-0.032137
Legendary,0.153396,0.501758,0.273620,0.345408,0.246377,0.448907,0.363937,0.326715,0.079794,1.000000,...,-0.014008,-0.043514,0.032686,0.001341,-0.083203,-0.056635,0.166625,0.008529,0.045761,-0.067244
Dragon,0.109370,0.196532,0.112416,0.208445,0.082088,0.149956,0.124341,0.103697,0.067775,0.219463,...,-0.041667,-0.063209,-0.041667,-0.035898,-0.076267,-0.038874,-0.056538,-0.049245,-0.038149,-0.082359


In [158]:
#In order to indentify which type is the strongest one I create a lookup table with the correlation to the total.
power_help = correlation_table[["Total",]]
power_help

,Total
Column1,1.000000
Total,1.000000
Sp. Atk,0.747250
Attack,0.736211
Sp. Def,0.717609
HP,0.618748
Defense,0.612787
Speed,0.575943
Legendary,0.501758
Dragon,0.196532


In [193]:
# I then merge the pokemon table with the power_help table and sort in descending order base on the correlation of Type 1.
# I then prepare a table with the
support_table = pd.merge(left = pokemon,
                                  right = power_help,
                                  how = 'left', 
                                  left_on = "Type 1", 
                                  right_index=True)
power_ranking = support_table[["Type 1","Total_y"]].sort_values("Total_y",ascending = False)
no_duplicates = power_ranking.drop_duplicates(subset='Type 1', keep="first")
no_duplicates['power_rank'] = no_duplicates['Total_y'].rank(ascending=False)
no_duplicates
# Based on following table we can say that the strongest type is the Dragon

C:\Users\EFFDF~1.SAN\AppData\Local\Temp/ipykernel_13708/151063110.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_duplicates['power_rank'] = no_duplicates['Total_y'].rank(ascending=False)


,Type 1,Total_y,power_rank
493,Dragon,0.196532,1.0
538,Psychic,0.094364,2.0
416,Steel,0.082000,3.0
437,Fire,0.050527,4.0
369,Rock,0.037524,5.0
790,Flying,0.029504,6.0
392,Dark,0.017818,7.0
262,Electric,0.016715,8.0
388,Ghost,0.007594,9.0
360,Ground,0.004082,10.0


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# If both Type 1 and Type 2 can be used, then I would choose Dragon and Psychic